# 주식 시계열 예측모델

전통적으로 대부분의 머신러닝(ML) 모델은 일부 관찰(샘플/예제)을 입력 피쳐로 사용하지만 데이터에 시간 차원은 없다.

시계열 예측 모형은 이전에 관측된 값을 기반으로 미래의 값을 예측할 수 있는 모형이다.

시계열 예측은 비정형 데이터에서 널리 사용된다. 평균 및 표준 편차와 같은 통계적 특성이 시간이 지남에 따라 일정하지 않은 데이터를 비정형 데이터라고 한다.

이러한 비정형 입력 데이터(해당 모델에 대한 입력으로 사용)를 일반적으로 시계열이라고 한다. 시계열의 예로는 시간 경과에 따른 온도, 주가, 주택 가격 등이 있다. 따라서 입력은 시간에 따라 연속적으로 나타나는 신호(시계열)이다.

시계열은 시간에 따라 순차적으로 취하는 일련의 관측치이다.

![image.png](https://i.imgur.com/IqyU5VO.png)

In [ ]:
import tensorflow as tf
from tensorflow import keras
# import tensorflow.keras as keras

import numpy as np
import pandas as pd

np.__version__, tf.__version__, keras.__version__

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline

In [ ]:
# 폰트 목록에서 폰트 찾기
for font in fm.fontManager.ttflist:
    if 'Nanum' in font.name:
        print(font.name, font.fname)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline

# font_path = 'C:/Windows/Fonts/NanumGothic.ttf'
# font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
font_path = "/Users/qkboo/Library/Fonts/NanumGothic.otf"
fontname = fm.FontProperties(fname=font_path, size=18).get_name()  # 폰트 패밀리 이름!

plt.rc('font', family=fontname)  #  'NanumGothic'
# plt.rcParams["font.family"] = fontname

plt.rcParams['axes.unicode_minus'] = False #glypy 8722: Axes에 - 표시 안되는 것
plt.title('한글 타이틀...')

## 데이터

구글 주가변동 데이터
 - http://finance.yahoo.com/quote/GOOG/history?ltr=1

### 야후 파이낸스에서 주가 데이터 가져오기

야후 파이낸스 덕분에 우리는 무료로 데이터를 얻을 수 있다. 다음 링크를 사용하여 테슬라의 주가 기록을 확인해보자.

https://finance.yahoo.com/quote/TSLA/history?period1=1436486400&period2=1594339200&interval=1d&filter=history&frequency=1d

![image.png](https://i.imgur.com/I37DVSw.png)

기간 동안의 주가를 다운로드 하면,

![image.png](https://i.imgur.com/RH53Tha.png)

###  데이터 처리

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# from keras.callbacks import EarlyStopping

#### Colab에 Drive 마운트

Google Drive에 `datasets` 폴더에 업로드 되었다고 가정.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! ls drive/MyDrive/datasets/

#### 데이터 읽기

In [ ]:
# df = pd.read_csv('drive/MyDrive/datasets/TSLA_2015-202207.csv')
df = pd.read_csv('TSLA_2015-202207.csv')
df.head()

In [ ]:
df.info()

일자별 종가(수정종가) 가격 그래프

In [ ]:
df.plot(x='Date', y='Adj Close', rot=25)

In [ ]:
# ndarray data

plt.plot(df['Adj Close'].values)

일자별 종가, 거래량 그래프

In [ ]:
df.plot(x='Date', y=['Adj Close', 'Volume'], logy=True, rot=25)

MinMax scaler 를 적용해서 종가/거래량 그래프를 그리면 2 변수의 추이를 보다 감각적으로 알 수 있다.

In [ ]:
scaler = MinMaxScaler()
tmp = scaler.fit_transform(df[['Adj Close', 'Volume']])
tmp

MinMax scaler 를 적용한 종가/거래량 데이터를 DataFrame 에 추가해 작업도 가능...

In [ ]:
_ = pd.DataFrame(tmp, index=df['Date'], columns=['Adj Close', 'Volume'])
_.plot(rot=25)

In [ ]:
# ndarray 와 date index
plt.plot(df['Date'], tmp)      #다변수
plt.legend(['Close','Volume'])
plt.show()

seaborn 을 사용한 종가 그래프

In [ ]:
import seaborn as sns

sns.lineplot(data=df, x='Date', y='Adj Close')

## 훈련/검증 세트 분리

시계열 데이터의 데이터셋은 보통 window_size라고 정의한다. window_size는 과거 기간의 주가 데이터에 기반하여 다음날의 종가를 예측할 것인가를 정하는 parameter이다. 과거 20일을 기반으로 내일 데이터를 예측한다라고 가정하면 window_size=20이다.

실제 100일의 과거 데이터를 기반으로 데이터셋을 분리하도록 한다.

In [ ]:
data = df[['Open','High','Low','Volume','Adj Close']].copy()
data.head()

스케일러의 시차가 1일(lag 1)인 입력 피쳐를 구축해 보자.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()#feature_range = (0, 1))
data_scaled = scaler.fit_transform(data)
data_scaled.shape

In [ ]:
SPLIT_RATE = int(df.shape[0] * .7)
WINDOW_SIZE = 60 # 시계열 주기.

In [ ]:
data_scaled[:SPLIT_RATE].shape, data_scaled[SPLIT_RATE:].shape

In [ ]:
train_scaled = data_scaled[:SPLIT_RATE]
test_scaled = data_scaled[SPLIT_RATE:]

In [ ]:
# def make_dataset(data:np.array, label, window=20):
#     X, y = [], []
    
#     for i in range(len(data) - window_size):
#         X.append(np.array(data[i:i+window_size]))
#         y.append(np.array(label[i+window_size]))
#     return np.array(X), np.array(y)

In [ ]:
def make_dataset(data:np.array, label, window=20):
    '''시계열 ndarray 로 전달된 data와 label 데이터를 window 만큼 간격으로 분리해 준다.
    결과는 (samples, steps, 1d) 형태로 반환한다.
    '''

    start_ = len(data)
    X = [data[i:i+window] for i in range(start_ - window)]
    y = [label[i+window] for i in range(start_ - window)]
    
    return np.array(X), np.array(y)

In [ ]:
train_set = train_scaled[:,:-1]
train_label = train_scaled[:,-1:]

In [ ]:
train_set.shape, train_label.shape

In [ ]:
X_train, y_train = make_dataset(train_set, train_label, WINDOW_SIZE)
X_train.shape, y_train.shape

In [ ]:
test_set = test_scaled[:,:-1]
test_label = test_scaled[:,-1:]

In [ ]:
test_set.shape, test_label.shape

In [ ]:
X_test, y_test = make_dataset(test_set, test_label, WINDOW_SIZE)
X_test.shape, y_test.shape

In [ ]:
# 윈도우 사이즈 원본 데이터와 훈련/검증 세트의 차이
X_train.shape[0] + X_test.shape[0], data_scaled.shape[0] - (X_train.shape[0] + X_test.shape[0])

In [ ]:
INPUT = X_train.shape[2]
INPUT

# RNN 학습

1. simple rnn

## SimpleRNN

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model1 = keras.Sequential()
model1.add(keras.layers.SimpleRNN(units=30, return_sequences=True, input_shape=(WINDOW_SIZE, INPUT)))
model1.add(keras.layers.SimpleRNN(30, activation='relu'))
model1.add(keras.layers.Dense(1))

In [ ]:

# 모델 학습과정 설정 
# model1.compile(optimizer=RMSprop(), loss='mae')
model1.compile(loss='mse', optimizer=RMSprop(), metrics=['mae'])


In [ ]:
%%time
history = model1.fit(X_train, y_train, 
                     epochs=100,
                     validation_split=0.2,
                     verbose=1)

In [ ]:
# LSTM 네트워크 학습 결과 확인
plt.title('RNN:LSTM')
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
%%time
# Adam
model1.compile(loss='mse', optimizer='adam', metrics=['mae'])

history = model1.fit(X_train, y_train, 
                     epochs=100,
                     validation_split=0.2,
                     verbose=1)

In [ ]:
# LSTM 네트워크 학습 결과 확인
plt.title('RNN:LSTM-adam')
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

## GRU 사용

In [ ]:
model_gru = keras.Sequential()
model_gru.add(keras.layers.GRU(32, input_shape=(WINDOW_SIZE, INPUT)))
model_gru.add(keras.layers.Dense(1))

In [ ]:
model_gru.compile(loss='mse', optimizer=RMSprop(), metrics=['mae'])

history = model_gru.fit(X_train, y_train, 
                     epochs=100,
                     validation_split=0.2,
                     verbose=0,
                     callbacks=[TqdmCallback()])

In [ ]:
# GRU 학습 결과 확인
plt.title('RNN::GRU')
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

## GRU에 드롭아웃 사용

훈련 손실과 검증 손실 곡선을 보면 모델이 과대적합인지 알 수 있다. 몇 번의 에포크 이후에 훈련 손실과 검증 손실이 현저하게 벌어지기 시작해서 이런 현상을 해결하기 위해 잘 알려진 드롭아웃을 적용해 보자.

In [ ]:
gru_dr = keras.Sequential()
gru_dr.add( keras.layers.GRU(32,
                              dropout=0.2,    # cuDNN을 사용할 수 없기 때문에
                              recurrent_dropout=0.2,
                              input_shape=(WINDOW_SIZE, INPUT)))
gru_dr.add(keras.layers.Dense(1))

In [ ]:
model_gru.compile(loss='mse', optimizer=RMSprop(), metrics=['mae'])

history = model_gru.fit(X_train, y_train, 
                     epochs=100,
                     steps_per_epoch=500,
                     validation_split=0.2,
                     verbose=0,
                     callbacks=[TqdmCallback()])

In [ ]:
# GRU 학습 결과 확인
plt.title('RNN::GRU DROPOUT')
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Stacking RNN

1. GRU Stacking
1. LSTM Stacking


<img src='https://miro.medium.com/max/994/1*27vV5Pit7XEwKHfi6s5Q1w.png'>
 - https://medium.com/geekculture/how-to-use-model-stacking-to-improve-machine-learning-predictions-d113278612d4

## GRU Stacking

In [ ]:
gru_stack = keras.Sequential()
gru_stack.add(keras.layers.GRU(32,
                     dropout=0.1,  # cuDNN을 사용할 수 없기 때문에
                     recurrent_dropout=0.5,
                     return_sequences=True,
                     input_shape=(WINDOW_SIZE, INPUT)))
gru_stack.add(keras.layers.GRU(64,
                     activation='relu',  # cuDNN을 사용할 수 없기 때문에
                     dropout=0.1,
                     recurrent_dropout=0.5
                    ))
gru_stack.add(keras.layers.Dense(1))


In [ ]:
gru_stack.summary()

In [ ]:
%%time
gru_stack.compile(loss='mse', optimizer=RMSprop(), metrics=['mae'])

history = gru_stack.fit(X_train, y_train, 
                     epochs=100,
                     validation_split=0.2,
                     verbose=1)

In [ ]:
# GRU 학습 결과 확인
plt.title('RNN::GRU Stacking')
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

## LSTM Stacking

우리는 50개의 뉴런과 4개의 숨겨진 층으로 LSTM을 만들 것이다. 마지막으로, 우리는 정규화된 주가를 예측하기 위해 출력층에 1개의 뉴런을 할당할 것이다. MSE 손실 함수와 Adam stochastic gradient decent optimizer를 사용할 것이다.

In [ ]:
# 입력으로 7일간의 5가지 데이터(시가, 종가, 고가, 저가, 거래량)을 다룬다.
lstmstack = keras.Sequential(name='LSTM1')
#Adding the first LSTM layer and some Dropout regularisation
lstmstack.add(keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (WINDOW_SIZE, INPUT)))
lstmstack.add(keras.layers.Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
lstmstack.add(keras.layers.LSTM(units = 50, return_sequences = True))
lstmstack.add(keras.layers.Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
lstmstack.add(keras.layers.LSTM(units = 50, return_sequences = True))
lstmstack.add(keras.layers.Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
lstmstack.add(keras.layers.LSTM(units = 50))
lstmstack.add(keras.layers.Dropout(0.2))
# Adding the output layer
lstmstack.add(keras.layers.Dense(units = 1))

lstmstack.summary()

학습 시작.

In [ ]:
# 모델 학습과정 설정 
lstmstack.compile(loss='mse', optimizer=RMSprop(), metrics=['mae'])

history = lstmstack.fit(X_train, y_train, 
                     epochs=100,
                     validation_split=0.2,
                     verbose=0,
                     callbacks=[TqdmCallback()])

In [ ]:
# LSTM 네트워크 학습 결과 확인
plt.title('RNN:LSTM Stack')
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# 모델 테스트
res = lstmstack.evaluate(X_test)
print('loss:', res[0], ', mae:', res[1])

In [ ]:
# 예측
predictions = lstmstack.predict(X_test)
predictions.shape

In [ ]:
y_test.shape, predictions.shape

In [ ]:
# Plot predictions
plt.plot(y_test[:100], 'bo', label='testY')              # Draw testY
plt.plot(predictions[:100], 'r', label='PredictY')    # Draw PredictY

plt.xlabel("Time Period")                   # axis x  labeling
plt.ylabel("Stock Price")                   # axis y  labeling

plt.title('LSTM Stack: Stock Prediction')               # Graph Title
plt.legend()                                # labeling for each graph
plt.show()                                  # show for us

In [ ]:
# predicted_stock_price = scaler.inverse_transform(predictions)
print( np.mean((predictions-y_test)**2))